<a href="https://colab.research.google.com/github/dineshkumarsarangapani/AlgoWiki/blob/gh-pages/MLP/case_study/bulldozer/Case_Study_Bulldozer_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset for this colab is available: [Training and validation set](https://drive.google.com/file/d/1Ji3oeXia4H4dd4bWpxkgTRfECS6Avn1d/view?usp=sharing) and [Test dataset](https://drive.google.com/file/d/1FWyWSnHNb28bit3mj2xrW5KpcvPKglGo/view?usp=sharing). 

The purpose of this study was to forecast the auction sale price of a specific piece of heavy equipment (a bulldozer) based on its usage, equipment type, and configuration. The information comes from auction result postings and covers use and equipment settings.
The RMSLE (root mean squared log error) between the actual and forecast auction prices is the competition's evaluation metric.

The data page contains links to sample submission files. Files submitted should be formatted as follows:

A newly created` header  `SalesID,SalePrice`;
has two columns:

`SalesID`: The validation set's SalesID in sorted order;
`SalePrice`: Your estimated sale price.

# [Root Mean Squared Log Error (RMSLE)](https://www.kaggle.com/code/carlolepelaars/understanding-the-metric-rmsle)

The formula for RMSLE is as follows:

$RMSLE=\sqrt{\frac{1}{n}\sum_{i=1}^{n}(log(p_i +1)-log(a_i +1))^2}$

where:

$n$  is the total number of observations in the (public/private) data set,

$p_i$  is the prediction of target, and

$a_i$  is the actual target for  $i$ .

$log(x)$  is the natural logarithm of  $x  ( log_e(x)$ .

#Prepare tools

##Random Forest Regression

Every decision tree has a significant variance, but when we mix all of them in parallel, the final variance is low since each decision tree is perfectly trained on that specific sample data, and so the outcome is dependent on numerous decision trees rather than one. In the case of a classification problem, the majority voting classifier is used to determine the final output. The final output of a regression problem is the mean of all the outputs. Aggregation is the name given to this section.

Random Forest is an ensemble technique that can handle both regression and classification tasks by combining many decision trees and a technique known as Bootstrap and Aggregation, or bagging. The core idea is to use numerous decision trees to determine the final output rather than depending on individual decision trees.

In [1]:
# import exploratory data analysis and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# models from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# model evaluations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#Inspect data

#Q1
Import the training and validation dataset `TrainAndValid.csv` with `low_memory=False`and priint first 5 rows of the dataset. What is the `auctionerrID` for `SalesID=1139246`? 



In [2]:
url ='https://github.com/dineshkumarsarangapani/Machine_learning_IITM/blob/main/MLP/case_study/bulldozer/TrainAndValidation.csv.zip?raw=true'


train_data = pd.read_csv(url, low_memory=False, compression='zip')

In [3]:
train_data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_data.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [5]:
train_data.shape

(412698, 53)

#Q2
How many non-null values are there in the state column?



In [6]:
train_data['state'].notnull().sum()

412698

In [7]:
train_data['state'].notnull()

0         True
1         True
2         True
3         True
4         True
          ... 
412693    True
412694    True
412695    True
412696    True
412697    True
Name: state, Length: 412698, dtype: bool

In [8]:
train_data['state'].count()

412698

# Which of the following columns have missing values?


In [9]:
train_data.isnull().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

In [10]:
train_data.sort_values('saledate', inplace=True)

In [11]:
train_data.loc[train_data['SalesID'] == 1646770]['saledate']

205615    1/17/1989 0:00
Name: saledate, dtype: object

In [12]:
train_data.loc[train_data['MachineID'] == 1194089]['YearMade']

274835    1980
Name: YearMade, dtype: int64

In [13]:
train_data_copy = train_data.copy()

In [14]:
train_data['saledate']

405675     1/1/2012 0:00
268962    1/10/2003 0:00
221519    1/10/2003 0:00
283617    1/10/2003 0:00
122970    1/10/2003 0:00
               ...      
19497      9/9/2010 0:00
103424     9/9/2010 0:00
19280      9/9/2010 0:00
19335      9/9/2010 0:00
22602      9/9/2011 0:00
Name: saledate, Length: 412698, dtype: object

Separate the saledate in saleYear,saleMonth,saleDay,saleDayOfWeek and saleDayOfYear and saledate column. Which numeric columns from the following options have null values

In [15]:
train_data['saledate']= pd.to_datetime(train_data['saledate'])

train_data['saleYear'] = train_data['saledate'].dt.year
train_data['saleYear']

405675    2012
268962    2003
221519    2003
283617    2003
122970    2003
          ... 
19497     2010
103424    2010
19280     2010
19335     2010
22602     2011
Name: saleYear, Length: 412698, dtype: int64

In [16]:
train_data['saleMonth'] = train_data['saledate'].dt.month
train_data['saleMonth']

405675    1
268962    1
221519    1
283617    1
122970    1
         ..
19497     9
103424    9
19280     9
19335     9
22602     9
Name: saleMonth, Length: 412698, dtype: int64

In [17]:
train_data['saleDay'] = train_data['saledate'].dt.day
train_data['saleDay']

405675     1
268962    10
221519    10
283617    10
122970    10
          ..
19497      9
103424     9
19280      9
19335      9
22602      9
Name: saleDay, Length: 412698, dtype: int64

In [18]:
train_data['saleDayOfWeek'] = train_data['saledate'].dt.dayofweek
train_data['saleDayOfWeek']

405675    6
268962    4
221519    4
283617    4
122970    4
         ..
19497     3
103424    3
19280     3
19335     3
22602     4
Name: saleDayOfWeek, Length: 412698, dtype: int64

In [19]:
train_data['saleDayOfYear'] = train_data['saledate'].dt.dayofyear
train_data['saleDayOfYear']

405675      1
268962     10
221519     10
283617     10
122970     10
         ... 
19497     252
103424    252
19280     252
19335     252
22602     252
Name: saleDayOfYear, Length: 412698, dtype: int64

In [20]:
train_data.isnull().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

In [21]:
train_data["state"] = pd.Categorical(train_data["state"]).codes
train_data["state"]

405675    21
268962    37
221519    37
283617    37
122970    37
          ..
19497     28
103424     4
19280     42
19335     43
22602      4
Name: state, Length: 412698, dtype: int8

In [22]:
train_data["UsageBand"] = pd.Categorical(train_data["UsageBand"]).codes
train_data["UsageBand"]

405675   -1
268962   -1
221519   -1
283617   -1
122970   -1
         ..
19497     1
103424   -1
19280     1
19335     0
22602     2
Name: UsageBand, Length: 412698, dtype: int8

In [23]:
train_data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfWeek,saleDayOfYear
405675,4449186,46173.2,2318824,26964,173,99.0,1996,0.0,-1,2012-01-01,...,None or Unspecified,None or Unspecified,None or Unspecified,NaN,NaN,2012,1,1,6,1
268962,1806686,8000.0,1077085,9549,132,2.0,1998,NaN,-1,2003-01-10,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10
221519,1691274,20500.0,1233251,22933,132,2.0,1994,NaN,-1,2003-01-10,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10
283617,1895558,9000.0,1458959,6054,132,2.0,1994,NaN,-1,2003-01-10,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10
122970,1467430,17500.0,1339914,4200,132,2.0,1988,NaN,-1,2003-01-10,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10


In [24]:
bool_cols = [col for col in train_data 
             if np.equal(len(train_data[col].dropna().unique()), 2)]
bool_cols

['Forks',
 'Stick',
 'Turbocharged',
 'Blade_Extension',
 'Engine_Horsepower',
 'Pushblock',
 'Scarifier',
 'Coupler_System',
 'Grouser_Tracks',
 'Track_Type',
 'Backhoe_Mounting']

In [25]:
train_data['Differential_Type'].isnull()

405675    True
268962    True
221519    True
283617    True
122970    True
          ... 
19497     True
103424    True
19280     True
19335     True
22602     True
Name: Differential_Type, Length: 412698, dtype: bool

In [26]:
missing_cols = train_data.columns[train_data.isnull().any()]
missing_cols

Index(['auctioneerID', 'MachineHoursCurrentMeter', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize', 'Drive_System',
       'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control', 'Stick',
       'Transmission', 'Turbocharged', 'Blade_Extension', 'Blade_Width',
       'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics', 'Pushblock',
       'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size', 'Coupler',
       'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [27]:

for col in missing_cols:
  train_data[col+'_is_missing'] = train_data[col].isnull()

In [28]:
train_data.dtypes

SalesID                           int64
SalePrice                       float64
MachineID                         int64
ModelID                           int64
datasource                        int64
                                 ...   
Backhoe_Mounting_is_missing        bool
Blade_Type_is_missing              bool
Travel_Controls_is_missing         bool
Differential_Type_is_missing       bool
Steering_Controls_is_missing       bool
Length: 97, dtype: object

In [29]:
category_cols = train_data.select_dtypes(include='category').columns
category_cols

Index([], dtype='object')

In [30]:
num_cols = train_data.select_dtypes(include=['number']).columns
num_cols

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'state', 'saleYear', 'saleMonth', 'saleDay', 'saleDayOfWeek',
       'saleDayOfYear'],
      dtype='object')

In [31]:
for col in category_cols:
  train_data[col] = pd.Categorical(train_data[col]).codes
  train_data[col]

In [32]:
for col in num_cols:
  train_data[col] = train_data[col].fillna(-1)
  train_data[col].head()

In [33]:
train_data[train_data['SalesID'] == 1646770]['Differential_Type_is_missing']

205615    True
Name: Differential_Type_is_missing, dtype: bool

In [34]:
train_data.isnull().sum()

SalesID                         0
SalePrice                       0
MachineID                       0
ModelID                         0
datasource                      0
                               ..
Backhoe_Mounting_is_missing     0
Blade_Type_is_missing           0
Travel_Controls_is_missing      0
Differential_Type_is_missing    0
Steering_Controls_is_missing    0
Length: 97, dtype: int64

In [35]:
training_set = train_data[train_data['saleYear'] < 2011]
training_set.shape

(365928, 97)

In [36]:
validation_set = train_data[train_data['saleYear'] >= 2011]
validation_set.shape

(46770, 97)